In [1]:
import os
from striprtf.striprtf import rtf_to_text
import sys
import json

In [2]:
import os
import sys  # sys.exit()를 사용하려면 import해야 합니다.

folder_path = "."  # <-- 여기에 원하는 폴더 경로를 입력하세요.

# 2. 지정된 폴더의 모든 파일 목록을 가져옵니다.
try:
    file_list = os.listdir(folder_path)

    # --- [수정] .rtf 파일만 필터링 ---
    # 1. 리스트 컴프리헨션을 사용해 .rtf (대소문자 무관) 파일만 새 리스트에 담습니다.
    rtf_files = [
        filename for filename in file_list if filename.lower().endswith(".rtf")
    ]

    # 2. 전체 목록 대신 필터링된 RTF 파일 목록을 출력합니다.
    print(rtf_files)
    # -------------------------------

except FileNotFoundError:
    print(f"오류: '{folder_path}' 폴더를 찾을 수 없습니다.")
    sys.exit()  # 프로그램 종료
except NotADirectoryError:
    print(f"오류: '{folder_path}'는 폴더가 아닙니다.")
    sys.exit()

['헌금자명단-2026-01-21(리모델링).RTF', '헌금자명단-2026-01-21.RTF']


In [3]:
# 셀 2: cp949 우선으로 안전하게 디코딩하는 유틸 함수
def decode_bytes_cp949(b: bytes, allow_fallback: bool = True) -> str:
    """
    bytes -> str 로 변환:
      1) 먼저 cp949로 시도
      2) 실패하면 (allow_fallback=True) utf-8, euc-kr, latin-1 순으로 시도
      3) 모두 실패하면 cp949로 errors='replace'하여 반환
    """
    try:
        return b.decode("cp949")
    except Exception:
        if not allow_fallback:
            raise
    # 폴백 시도
    for enc in ("utf-8", "euc-kr", "latin-1"):
        try:
            return b.decode(enc)
        except Exception:
            continue
    # 최후의 수단: cp949로 강제 디코드(손실 허용)
    return b.decode("cp949", errors="replace")

In [4]:
text = ""

# 셀 4: 수정된 파일 처리 루프
for filename in rtf_files:
    # if not filename.lower().endswith(".rtf"): # (file_list 생성 시 미리 필터링)
    #     continue

    full_path = os.path.join(folder_path, filename)
    print(f"\n--- [{filename}] 파일 텍스트 추출 시작 ---")

    try:
        # 1) 바이너리로 읽기
        with open(full_path, "rb") as f:
            rtf_content = f.read()

        # 2) bytes인지 확인하고 cp949로 디코딩
        if isinstance(rtf_content, bytes):
            rtf_str = decode_bytes_cp949(rtf_content, allow_fallback=True)
        else:
            rtf_str = rtf_content

        # 3) striprtf의 rtf_to_text 함수를 사용 (이 함수가 \par을 \n으로 변환)
        current_file_text = rtf_to_text(rtf_str)

        # 4) (수정) 현재 '파일 1개'의 결과만 출력 (누적X)
        print("--- [추출된 텍스트 미리보기] ---")
        preview_len = 1000
        if len(current_file_text) > preview_len:
            print(current_file_text[:preview_len] + "\n\n... (출력 생략) ...")
        else:
            print(current_file_text)

        text += current_file_text

    except FileNotFoundError:
        print(f"오류: '{filename}' 파일을 찾을 수 없습니다.")
    except Exception as e:
        print(f"'{filename}' 처리 중 오류 발생: {e}")
        traceback.print_exc()

    print(f"--- [{filename}] 추출 완료 ---\n")

print("모든 RTF 파일 처리가 완료되었습니다.")

# (선택) 모든 파일의 텍스트를 누적한 최종 결과 확인
# print("\n--- [모든 파일 누적 결과] ---")
# print(all_files_text)


--- [헌금자명단-2026-01-21(리모델링).RTF] 파일 텍스트 추출 시작 ---
--- [추출된 텍스트 미리보기] ---
■ 기간 : 2026.1.18 ~ 2026.1.18  ■ 항목 : 1-특별계정 ■ 인원 : 1 명--------------------------------------------------------------------------------1-10 리프트1-11 건물리모델링 * 1 명    이점례                1-12 헌물--------------------------------------------------------------------------------■ 총합계 : 100,000원
--- [헌금자명단-2026-01-21(리모델링).RTF] 추출 완료 ---


--- [헌금자명단-2026-01-21.RTF] 파일 텍스트 추출 시작 ---
--- [추출된 텍스트 미리보기] ---
■ 기간 : 2026.1.18 ~ 2026.1.18  ■ 항목 : 1-일반계정 ■ 인원 : 189 명--------------------------------------------------------------------------------유초등부중고등부주일헌금 * 1 명    주일헌금              십일조 * 12 명    김창기                박정자                손순용                유명자                이금범                이상례                이호석                익명                  정순희                주시문                최면식김민숙          황호순                감사헌금 * 51 명    강경애                강성호                강원구김미소          김동영                김신충                김영권  

In [5]:
print(text)

■ 기간 : 2026.1.18 ~ 2026.1.18  ■ 항목 : 1-특별계정 ■ 인원 : 1 명--------------------------------------------------------------------------------1-10 리프트1-11 건물리모델링 * 1 명    이점례                1-12 헌물--------------------------------------------------------------------------------■ 총합계 : 100,000원■ 기간 : 2026.1.18 ~ 2026.1.18  ■ 항목 : 1-일반계정 ■ 인원 : 189 명--------------------------------------------------------------------------------유초등부중고등부주일헌금 * 1 명    주일헌금              십일조 * 12 명    김창기                박정자                손순용                유명자                이금범                이상례                이호석                익명                  정순희                주시문                최면식김민숙          황호순                감사헌금 * 51 명    강경애                강성호                강원구김미소          김동영                김신충                김영권                김진선                김혜정                나기대                나아린                나지환                나충남                비토                  손순용                안소영                양

In [6]:
import re

pattern = r"\d{4}\.\d{1,2}\.\d{1,2}"

# 1. 텍스트에서 패턴을 검색(search)합니다.
match = re.search(pattern, text)

# 2. 매칭된 결과가 있는지 확인하고 출력합니다.
if match:
    date_str = match.group(0)  # (match.group()과 동일)
    date = date_str
    print(date_str)
else:
    print("날짜 패턴을 찾지 못했습니다.")

2026.1.18


In [7]:
pattern = r"-{5,}"
parts = re.split(pattern, text)

# parts의 길이를 확인하여 5개로 잘 나뉘었는지 확인
if len(parts) == 5:

    # 2번째 부분 (인덱스 1)
    part_2 = parts[1].strip()

    # 4번째 부분 (인덱스 3)
    part_4 = parts[3].strip()

    print("--- 2번째 부분 (parts[1]) ---")
    print(part_2)
    print("\n--- 4번째 부분 (parts[3]) ---")
    print(part_4)

    # 만약 두 텍스트를 합치고 싶다면
    combined_text = part_2 + "\n" + part_4
    # print("\n--- 합친 결과 ---")
    # print(combined_text)

else:
    print(f"문자열이 5개 부분으로 나뉘지 않았습니다. (총 {len(parts)}개 부분)")
    combined_text = ""

--- 2번째 부분 (parts[1]) ---
1-10 리프트1-11 건물리모델링 * 1 명    이점례                1-12 헌물

--- 4번째 부분 (parts[3]) ---
유초등부중고등부주일헌금 * 1 명    주일헌금              십일조 * 12 명    김창기                박정자                손순용                유명자                이금범                이상례                이호석                익명                  정순희                주시문                최면식김민숙          황호순                감사헌금 * 51 명    강경애                강성호                강원구김미소          김동영                김신충                김영권                김진선                김혜정                나기대                나아린                나지환                나충남                비토                  손순용                안소영                양승찬                유미종                유유덕                유진경                이금범                이금형                이승제                이실                  이은신                이재영                이종록주지연          이효탁                이흥석                익명                  임남희                전숙자                전예은       

In [8]:
combined_text = combined_text.replace("첫 열 매", "첫열매")
print(combined_text)

1-10 리프트1-11 건물리모델링 * 1 명    이점례                1-12 헌물
유초등부중고등부주일헌금 * 1 명    주일헌금              십일조 * 12 명    김창기                박정자                손순용                유명자                이금범                이상례                이호석                익명                  정순희                주시문                최면식김민숙          황호순                감사헌금 * 51 명    강경애                강성호                강원구김미소          김동영                김신충                김영권                김진선                김혜정                나기대                나아린                나지환                나충남                비토                  손순용                안소영                양승찬                유미종                유유덕                유진경                이금범                이금형                이승제                이실                  이은신                이재영                이종록주지연          이효탁                이흥석                익명                  임남희                전숙자                전예은                정금순                정문수                정민수   

In [9]:
import re  # 정규 표현식 모듈 임포트


def parse_offerings(text):
    """
    헌금 내역 텍스트를 파싱하여 딕셔너리로 반환합니다.
    """
    result_dict = {}  # 결과를 담을 빈 딕셔너리
    current_key = None  # 현재 처리 중인 항목(십일조, 감사헌금 등)

    # 1. 텍스트를 공백(스페이스, 탭, 줄바꿈) 기준으로 모두 나눕니다.
    #    '김영권 황남순'처럼 이름에 공백이 있는 경우를 처리하기 위해
    #    우선 '*'가 포함된 항목을 먼저 분리합니다.

    # 정규 표현식을 사용해 '항목 * X명' 부분을 찾아 분리합니다.
    # (re.split)은 괄호로 묶인 패턴(항목명)도 결과에 포함시킵니다.
    parts = re.split(r"\s*(\S+\s\*\s\d+\s명)\s*", text.strip())

    for part in parts:
        if not part:  # 빈 문자열은 건너뜁니다.
            continue

        # 2. 항목(Key)인지 이름(Value)인지 확인
        if "*" in part and "명" in part:
            # 항목인 경우 (예: '십일조 * 5명')
            # '*' 앞부분을 key로 사용합니다. (공백 제거)
            current_key = part.split("*")[0].strip()
            result_dict[current_key] = []  # 딕셔너리에 새 리스트 생성
        elif current_key:
            # 이름인 경우 (앞에서 current_key가 설정되었다면)
            # 이름들이 여러 공백으로 나열되어 있으므로,
            # 공백이 2칸 이상인 경우를 기준으로 이름을 분리합니다.
            # '김영권 황남순'처럼 이름 사이의 공백 1칸은 유지됩니다.
            names = re.split(r"\s{2,}", part.strip())  # 2칸 이상 공백으로 분리

            # 분리된 이름 리스트를 결과 딕셔너리에 추가
            result_dict[current_key].extend(names)

    return result_dict


# 함수 실행 및 결과 출력
offering_dict = parse_offerings(combined_text)
print(offering_dict)

{'건물리모델링': ['이점례', '1-12 헌물'], '유초등부중고등부주일헌금': ['주일헌금'], '십일조': ['김창기', '박정자', '손순용', '유명자', '이금범', '이상례', '이호석', '익명', '정순희', '주시문', '최면식김민숙', '황호순'], '감사헌금': ['강경애', '강성호', '강원구김미소', '김동영', '김신충', '김영권', '김진선', '김혜정', '나기대', '나아린', '나지환', '나충남', '비토', '손순용', '안소영', '양승찬', '유미종', '유유덕', '유진경', '이금범', '이금형', '이승제', '이실', '이은신', '이재영', '이종록주지연', '이효탁', '이흥석', '익명', '임남희', '전숙자', '전예은', '정금순', '정문수', '정민수', '정순희', '정예진', '정용분', '정윤일', '정행웅', '조기순', '조소연', '조점숙', '조항남조진현', '주시문', '채졍윤', '채훈병', '최성옥', '최정욱심은형', '최희옥', '한상덕'], '속회헌금유치부청년부신년감사': ['익명', '황호순'], '부활절헌금맥추절헌금추수감사헌금성탄절헌금선교헌금': ['익명', '황호순'], '서원헌금꽃꽃이헌금': ['김준호문재숙', '황남순'], '장학헌금첫열매금식헌금성전봉헌헌금': ['강경애', '강원구 김미소', '김기철', '김동현', '김모세', '김성환 손지혜', '김승희', '김신충', '김영권 황남순', '김용옥', '김이분', '김주엘', '김준호', '김창기', '나아란', '나지환', '나충남', '도광수', '박정자', '배신봉 임윤정', '벡선규', '손순용', '손용국', '송의섭', '심영식', '양승찬', '염계현 정금순', '유명자', '유유덕', '유정휘', '윤광식', '윤연숙', '응암교회', '이광희', '이금형', '이승자 손종희', '이재영', '이종록 주지연', '이창은', '이효탁', '이흥석', '익명', '임남희', '정세영', '정연순'

In [10]:
def rename_combined_keys(d):

    keys_to_rename = {}  # { '이전 키': '새 키' } 형태로 저장
    pattern = re.compile(r"(.*?)(헌금|번제|첫열매|건물리모델링|설립)")
    for old_key in d.keys():
        matches = pattern.findall(old_key)
        if len(matches) > 1:
            last_match = matches[-1]
            new_key = "".join(last_match)
            if new_key in d and new_key != old_key:
                print(
                    f"경고: 키 '{new_key}'가 중복됩니다. 데이터 병합이 필요할 수 있습니다."
                )
            keys_to_rename[old_key] = new_key

    new_dict = d.copy()  # 원본 복사
    for old_key, new_key in keys_to_rename.items():
        if new_key in new_dict:
            new_dict[new_key].extend(new_dict[old_key])
        else:
            new_dict[new_key] = new_dict[old_key]
        new_dict.pop(old_key)

    return new_dict


dict1 = rename_combined_keys(offering_dict)

In [11]:
for key, value in dict1.items():
    print(f"{key}: {value}")

건물리모델링: ['이점례', '1-12 헌물']
유초등부중고등부주일헌금: ['주일헌금']
십일조: ['김창기', '박정자', '손순용', '유명자', '이금범', '이상례', '이호석', '익명', '정순희', '주시문', '최면식김민숙', '황호순']
감사헌금: ['강경애', '강성호', '강원구김미소', '김동영', '김신충', '김영권', '김진선', '김혜정', '나기대', '나아린', '나지환', '나충남', '비토', '손순용', '안소영', '양승찬', '유미종', '유유덕', '유진경', '이금범', '이금형', '이승제', '이실', '이은신', '이재영', '이종록주지연', '이효탁', '이흥석', '익명', '임남희', '전숙자', '전예은', '정금순', '정문수', '정민수', '정순희', '정예진', '정용분', '정윤일', '정행웅', '조기순', '조소연', '조점숙', '조항남조진현', '주시문', '채졍윤', '채훈병', '최성옥', '최정욱심은형', '최희옥', '한상덕']
속회헌금유치부청년부신년감사: ['익명', '황호순']
일천번제: ['김동영', '김은지', '김주찬', '김준호', '동산유치원', '손용국', '유진경', '윤성구', '이승제', '주성우', '황화자']
부흥회헌금(특별성회): ['강경애', '강윤정', '김동영', '김민숙', '김신충', '김영권', '김이분', '김정현', '김준호', '김진선', '김진선', '나충남', '문재숙', '박건규', '박경자', '박정자', '손순용', '손용국', '오순진', '유미종', '유유덕', '유진경', '윤성구', '이계연', '이기환', '이무영', '이상선', '이승제', '이실', '이재영', '이종목', '익명', '임남희', '임봉숙', '정금순', '정윤일', '조기순', '조순희', '조점숙', '조항남 조진현', '주성우', '주지연', '최정욱 심은형', '최정훈', '최호석', '최희옥', '한경아', '황남순', '부스러기헌금교육주일헌금

In [12]:
# 2. 삭제할 단어 목록
substrings_to_remove = ["주일헌금", "특별헌금", "공과금", "구제헌금", "헌물"]

# 3. 삭제할 키를 임시 저장할 리스트
keys_to_delete = []

# [★핵심] list(dict1.items())로 딕셔너리의 복사본을 순회합니다.
# (이렇게 해야 순회 중에 원본 딕셔너리를 수정할 수 있습니다.)
for key, value_list in list(dict1.items()):

    key_is_bad = False  # key 자체가 삭제 대상인지 확인

    # --- 1단계: Key 검사 ---
    for substring in substrings_to_remove:
        if substring in key:
            keys_to_delete.append(key)  # key를 삭제 목록에 추가
            key_is_bad = True
            break  # 이 key는 검사 종료

    # --- 2단계: Item 검사 (Key가 삭제 대상이 아닐 경우에만) ---
    if not key_is_bad:
        # value_list (예: ['이영희', '특별헌금(익명)', '최장로'])를 검사

        # [★핵심] 리스트 컴프리헨션으로 '안전한' 항목만 골라 새 리스트 생성
        # any(...) : '주일헌금'이나 '특별헌금' 중 *어느 하나라도* item에 있으면 True
        safe_items = [
            item
            for item in value_list
            if not any(substring in item for substring in substrings_to_remove)
        ]

        # 딕셔너리의 값을 '안전한' 항목만 담긴 새 리스트로 교체
        dict1[key] = safe_items

# --- 3단계: 1단계에서 찾은 Key들을 실제로 삭제 ---
for key in keys_to_delete:
    dict1.pop(key, None)

# 4. 최종 결과 출력
print("\n--- ✅ 삭제 후 결과 ---")
print(json.dumps(dict1, indent=4, ensure_ascii=False))


--- ✅ 삭제 후 결과 ---
{
    "건물리모델링": [
        "이점례"
    ],
    "십일조": [
        "김창기",
        "박정자",
        "손순용",
        "유명자",
        "이금범",
        "이상례",
        "이호석",
        "익명",
        "정순희",
        "주시문",
        "최면식김민숙",
        "황호순"
    ],
    "감사헌금": [
        "강경애",
        "강성호",
        "강원구김미소",
        "김동영",
        "김신충",
        "김영권",
        "김진선",
        "김혜정",
        "나기대",
        "나아린",
        "나지환",
        "나충남",
        "비토",
        "손순용",
        "안소영",
        "양승찬",
        "유미종",
        "유유덕",
        "유진경",
        "이금범",
        "이금형",
        "이승제",
        "이실",
        "이은신",
        "이재영",
        "이종록주지연",
        "이효탁",
        "이흥석",
        "익명",
        "임남희",
        "전숙자",
        "전예은",
        "정금순",
        "정문수",
        "정민수",
        "정순희",
        "정예진",
        "정용분",
        "정윤일",
        "정행웅",
        "조기순",
        "조소연",
        "조점숙",
        "조항남조진현",
        "주시문",
        "채졍윤",
        "채훈병",
        "최성옥",
 

In [13]:
for key, value in dict1.items():
    print(f"{key}: {value}")

건물리모델링: ['이점례']
십일조: ['김창기', '박정자', '손순용', '유명자', '이금범', '이상례', '이호석', '익명', '정순희', '주시문', '최면식김민숙', '황호순']
감사헌금: ['강경애', '강성호', '강원구김미소', '김동영', '김신충', '김영권', '김진선', '김혜정', '나기대', '나아린', '나지환', '나충남', '비토', '손순용', '안소영', '양승찬', '유미종', '유유덕', '유진경', '이금범', '이금형', '이승제', '이실', '이은신', '이재영', '이종록주지연', '이효탁', '이흥석', '익명', '임남희', '전숙자', '전예은', '정금순', '정문수', '정민수', '정순희', '정예진', '정용분', '정윤일', '정행웅', '조기순', '조소연', '조점숙', '조항남조진현', '주시문', '채졍윤', '채훈병', '최성옥', '최정욱심은형', '최희옥', '한상덕']
속회헌금유치부청년부신년감사: ['익명', '황호순']
일천번제: ['김동영', '김은지', '김주찬', '김준호', '동산유치원', '손용국', '유진경', '윤성구', '이승제', '주성우', '황화자']
부흥회헌금(특별성회): ['강경애', '강윤정', '김동영', '김민숙', '김신충', '김영권', '김이분', '김정현', '김준호', '김진선', '김진선', '나충남', '문재숙', '박건규', '박경자', '박정자', '손순용', '손용국', '오순진', '유미종', '유유덕', '유진경', '윤성구', '이계연', '이기환', '이무영', '이상선', '이승제', '이실', '이재영', '이종목', '익명', '임남희', '임봉숙', '정금순', '정윤일', '조기순', '조순희', '조점숙', '조항남 조진현', '주성우', '주지연', '최정욱 심은형', '최정훈', '최호석', '최희옥', '한경아', '황남순']
선교헌금: ['익명', '황호순']
꽃꽃이헌금: ['김준호문재숙', '황남순']
성전

In [14]:
new_dict = {f"{key} ({len(value)}명)": value for key, value in dict1.items()}
new_dict["날짜"] = str(date)
# 3. 결과 출력
import json

print(json.dumps(new_dict, indent=4, ensure_ascii=False))

{
    "건물리모델링 (1명)": [
        "이점례"
    ],
    "십일조 (12명)": [
        "김창기",
        "박정자",
        "손순용",
        "유명자",
        "이금범",
        "이상례",
        "이호석",
        "익명",
        "정순희",
        "주시문",
        "최면식김민숙",
        "황호순"
    ],
    "감사헌금 (51명)": [
        "강경애",
        "강성호",
        "강원구김미소",
        "김동영",
        "김신충",
        "김영권",
        "김진선",
        "김혜정",
        "나기대",
        "나아린",
        "나지환",
        "나충남",
        "비토",
        "손순용",
        "안소영",
        "양승찬",
        "유미종",
        "유유덕",
        "유진경",
        "이금범",
        "이금형",
        "이승제",
        "이실",
        "이은신",
        "이재영",
        "이종록주지연",
        "이효탁",
        "이흥석",
        "익명",
        "임남희",
        "전숙자",
        "전예은",
        "정금순",
        "정문수",
        "정민수",
        "정순희",
        "정예진",
        "정용분",
        "정윤일",
        "정행웅",
        "조기순",
        "조소연",
        "조점숙",
        "조항남조진현",
        "주시문",
        "채졍윤",
        "채훈병",
        "최성옥",
   

In [15]:
priority_keys = ["날짜", "십일조", "감사헌금", "일천번제"]

# 2. 순서를 적용할 새 딕셔너리
ordered_dict = {}

# --- [★ 이 부분이 수정되었습니다] ---
# [1단계] 우선순위 키부터 새 딕셔너리에 추가
for key in priority_keys:
    # (예: key = '십일조')

    # new_dict의 모든 실제 키(dict_key)와 값(value)을 순회합니다.
    for dict_key, value in new_dict.items():
        # (예: dict_key = '십일조 (27명)')

        # [수정] '십일조'가 '십일조 (27명)' 문자열 안에 포함(in)되어 있는지 확인
        if key in dict_key:

            # (중요) 이미 다른 우선순위 키에 의해 추가되지 않았는지 확인
            if dict_key not in ordered_dict:
                ordered_dict[dict_key] = value

# [2단계] (필수) 나머지 키들을 새 딕셔너리에 추가
# (예: '선교헌금 (5명)' 등)
for dict_key, value in new_dict.items():
    if dict_key not in ordered_dict:
        ordered_dict[dict_key] = value

In [16]:
for key, value in ordered_dict.items():
    print(f"{key}: {value}")

날짜: 2026.1.18
십일조 (12명): ['김창기', '박정자', '손순용', '유명자', '이금범', '이상례', '이호석', '익명', '정순희', '주시문', '최면식김민숙', '황호순']
감사헌금 (51명): ['강경애', '강성호', '강원구김미소', '김동영', '김신충', '김영권', '김진선', '김혜정', '나기대', '나아린', '나지환', '나충남', '비토', '손순용', '안소영', '양승찬', '유미종', '유유덕', '유진경', '이금범', '이금형', '이승제', '이실', '이은신', '이재영', '이종록주지연', '이효탁', '이흥석', '익명', '임남희', '전숙자', '전예은', '정금순', '정문수', '정민수', '정순희', '정예진', '정용분', '정윤일', '정행웅', '조기순', '조소연', '조점숙', '조항남조진현', '주시문', '채졍윤', '채훈병', '최성옥', '최정욱심은형', '최희옥', '한상덕']
일천번제 (11명): ['김동영', '김은지', '김주찬', '김준호', '동산유치원', '손용국', '유진경', '윤성구', '이승제', '주성우', '황화자']
건물리모델링 (1명): ['이점례']
속회헌금유치부청년부신년감사 (2명): ['익명', '황호순']
부흥회헌금(특별성회) (48명): ['강경애', '강윤정', '김동영', '김민숙', '김신충', '김영권', '김이분', '김정현', '김준호', '김진선', '김진선', '나충남', '문재숙', '박건규', '박경자', '박정자', '손순용', '손용국', '오순진', '유미종', '유유덕', '유진경', '윤성구', '이계연', '이기환', '이무영', '이상선', '이승제', '이실', '이재영', '이종목', '익명', '임남희', '임봉숙', '정금순', '정윤일', '조기순', '조순희', '조점숙', '조항남 조진현', '주성우', '주지연', '최정욱 심은형', '최정훈', '최호석', '최희옥', '한경아', '황남순']

In [17]:
import json

filename = "my_data.txt"

# --- 1. 딕셔너리를 TXT 파일로 저장하기 ---
try:
    with open(filename, "w", encoding="utf-8") as f:
        # json.dump()를 사용하여 딕셔너리를 파일에 텍스트로 저장
        # ensure_ascii=False: 한글을 깨지지 않게 저장
        # indent=4: 보기 좋게 4칸 들여쓰기 (선택 사항)
        json.dump(ordered_dict, f, ensure_ascii=False, indent=4)

    print(f"✅ 딕셔너리를 '{filename}' 파일로 저장했습니다.")

except Exception as e:
    print(f"🚨 저장 중 오류 발생: {e}")


# --- 2. TXT 파일을 다시 딕셔너리로 불러오기 ---
try:
    with open(filename, "r", encoding="utf-8") as f:
        # json.load()를 사용하여 파일의 텍스트를 딕셔너리로 불러오기
        loaded_dict = json.load(f)

    print(f"\n✅ '{filename}' 파일에서 딕셔너리를 불러왔습니다.")
    print("--- 불러온 딕셔너리 내용 ---")
    print(loaded_dict)

    # 타입 확인
    print(f"\n불러온 데이터 타입: {type(loaded_dict)}")

except FileNotFoundError:
    print(f"🚨 불러오기 오류: '{filename}' 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"🚨 불러오기 중 오류 발생: {e}")

✅ 딕셔너리를 'my_data.txt' 파일로 저장했습니다.

✅ 'my_data.txt' 파일에서 딕셔너리를 불러왔습니다.
--- 불러온 딕셔너리 내용 ---
{'날짜': '2026.1.18', '십일조 (12명)': ['김창기', '박정자', '손순용', '유명자', '이금범', '이상례', '이호석', '익명', '정순희', '주시문', '최면식김민숙', '황호순'], '감사헌금 (51명)': ['강경애', '강성호', '강원구김미소', '김동영', '김신충', '김영권', '김진선', '김혜정', '나기대', '나아린', '나지환', '나충남', '비토', '손순용', '안소영', '양승찬', '유미종', '유유덕', '유진경', '이금범', '이금형', '이승제', '이실', '이은신', '이재영', '이종록주지연', '이효탁', '이흥석', '익명', '임남희', '전숙자', '전예은', '정금순', '정문수', '정민수', '정순희', '정예진', '정용분', '정윤일', '정행웅', '조기순', '조소연', '조점숙', '조항남조진현', '주시문', '채졍윤', '채훈병', '최성옥', '최정욱심은형', '최희옥', '한상덕'], '일천번제 (11명)': ['김동영', '김은지', '김주찬', '김준호', '동산유치원', '손용국', '유진경', '윤성구', '이승제', '주성우', '황화자'], '건물리모델링 (1명)': ['이점례'], '속회헌금유치부청년부신년감사 (2명)': ['익명', '황호순'], '부흥회헌금(특별성회) (48명)': ['강경애', '강윤정', '김동영', '김민숙', '김신충', '김영권', '김이분', '김정현', '김준호', '김진선', '김진선', '나충남', '문재숙', '박건규', '박경자', '박정자', '손순용', '손용국', '오순진', '유미종', '유유덕', '유진경', '윤성구', '이계연', '이기환', '이무영', '이상선', '이승제', '이실', '이재영', '이종목', '익명', '임남희',